<a href="https://colab.research.google.com/github/KhushiBhambri/YogNet/blob/55_Khushi/PoseStreamModel/FinalPoseStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# RGBmodel = load_model('RGBStream.h5')  
# Posemodel = load_model('PoseStream.h5')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Major_Project_Cse"

Mounted at /content/drive
/content/drive/MyDrive/Major_Project_Cse


In [2]:
!pip install mediapipe
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8,model_complexity=2)

# Setup the Pose function for videos - for video processing.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def mpPoseImg(image_pose, pose=pose_image, draw=False, display=False):
    
    annotated_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
 
    
    results = pose.process(image_in_RGB)

    if results.pose_landmarks and draw:    
      mp_drawing.draw_landmarks(image=annotated_image,landmark_list=results.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


        # mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
        #                           landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),thickness=3, circle_radius=3),
        #                           connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),thickness=2, circle_radius=2))

    if display:
            
            # plt.figure(figsize=[22,22])
            # plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            # plt.subplot(122);plt.imshow(annotated_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');
            cv2.imshow(annotated_image)
            return results

    else:
        return annotated_image,results
     


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 16.0 MB/s eta 0:00:00


In [12]:
numKeypoints=35
BODY_PARTS = {
    0: 'nose', 1: 'left_eye_inner', 2: 'left_eye', 3: 'left_eye_outer', 4: 'right_eye_inner', 5: 'right_eye', 6: 'right_eye_outer', 7: 'left_ear', 8: 'right_ear',
    9: 'mouth_left', 10: 'mouth_right', 11: 'left_shoulder', 12: 'right_shoulder', 13: 'left_elbow', 14: 'right_elbow', 15: 'left_wrist',16: 'right_wrist',
    17: 'left_pinky_finger', 18: 'right_pinky_finger', 19: 'left_index_finger', 20: 'right_index_finger', 21: 'left_thumb', 22: 'right_thumb', 23: 'left_hip', 24: 'right_hip',
    25: 'left_knee', 26: 'right_knee', 27: 'left_ankle', 28: 'right_ankle', 29: 'left_heel', 30: 'right_heel', 31: 'left_foot_index', 32: 'right_foot_index', 33: 'mid_hip', 34: 'neck'
}

In [5]:
# Define the groups of keypoints by body parts
keypoint_groups = {
     0:[1,2,3,4,5,6,7,8],
     1:[0,2,3,4,5,6],
     2:[0,1,3,4,5,6],
     3:[0,1,2,4,5,6],
     4:[0,1,2,3,5,6],
     5:[0,1,2,3,4,6],
     6:[0,1,2,3,4,5],
     7:[0,1,2,3,9],
     8:[0,4,5,6,10],
     9:[0,7,10],
     10:[0,8,9],
     11:[12,13,9],
     12:[11,14,10],
     13:[11,15],
     14:[12,16],
     15:[13,17,19],
     16:[14,18,20],
     17:[15,19,21],
     18:[16,20,22],
     19:[15,17,21],
     20:[16,18,22],
     21:[17,19],
     22:[18,20],
     23:[24,25],
     24:[23,26],
     25:[23,27],
     26:[24,28],
     27:[25,29,31],
     28:[26,30,32],
     29:[27,31],
     30:[28,32],
     31:[27,29],
     32:[28,30],
     33:[23,24],
     34:[11,12]
}


In [6]:
def ReplacebyAverage(index,keypoints):
  group_indices = keypoint_groups[index]
  group_keypoints = [keypoints[i] for i in group_indices]
  valid_keypoints = [k for k in group_keypoints if k is not None]
  if len(valid_keypoints) > 0:
     group_average_x = np.mean([k[0] for k in valid_keypoints if k[0] is not None])
     group_average_y = np.mean([k[1] for k in valid_keypoints if k[1] is not None])
    #  keypoints[index][0]=group_average_x
    #  keypoints[index][1]=group_average_y
  else:
    # If all keypoints in the group are missing, replace with (0,0) or another default value
    group_average_x, group_average_y = 0, 0
  print(group_average_x,group_average_y)
  return [group_average_x,group_average_y]



In [7]:
def ReplaceNonebyKeyAverage(frame):
  for i in range (numKeypoints):
    if(frame[i][0] is None or frame[i][1] is None):
       group_indices = keypoint_groups[i]
       group_keypoints = [frame[j] for j in group_indices if (frame[j][0] is not None and frame[j][1] is not None)]
       if len(group_keypoints) > 0:
        frame[i][0] = np.mean([k[0] for k in group_keypoints if k[0] is not None])
        frame[i][1] = np.mean([k[1] for k in group_keypoints if k[1] is not None])
       else:
        frame[i][0] = 0.0
        frame[i][1] = 0.0
  return frame

In [8]:
# calculate the center of the object in each frame
def CentralisedKeypointswithLocalAverages(keypoints):
  frame_centers = []
 
  for frame_keypoints in keypoints:
    # assume the center is the mean of all keypoints
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))

  relative_keypoints = keypoints

  for j in range(len(relative_keypoints)):               #frame no
    for k in range(len(relative_keypoints[j])):          # each keypoint of frame
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]

  for i in range(len(relative_keypoints)):   #each frame
       relative_keypoints[i]=ReplaceNonebyKeyAverage(relative_keypoints[i])
    
  return relative_keypoints



In [27]:
def ReplaceNonebyFrameAverage(window,NoneFrames):
  # averageFrame = [0.0,0.0]*numKeypoints
  averageFrame = [[0.0,0.0] for _ in range(numKeypoints)]
  new_window = [] 
  numFrames = 16-len(NoneFrames)
  for i in range(len(window)):  #ith Frame of window
    if(i not in NoneFrames):
      # print(len(averageFrame))
      for j in range(len(window[i])):   #jth Keypoint of ith Frame
        averageFrame[j][0]+=window[i][j][0]
        averageFrame[j][1]+=window[i][j][1]
      averageFrame[j][0]/=numFrames
      averageFrame[j][1]/=numFrames
  print(averageFrame)
  for i in range(len(window)):  #ith Frame of window
      if(i in NoneFrames):
        new_window.append(averageFrame)
      else:
        new_window.append(window[i])

  return new_window

In [26]:
# calculate the center of the object in each frame
def CentralisedKeypointswithGlobalAverages(window):
  frame_centers = []
  for frame_keypoints in window:
    # assume the center is the mean of all window
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))

  relative_keypoints = window

  for j in range(len(relative_keypoints)):               #frame no
    for k in range(len(relative_keypoints[j])):          # each keypoint of frame
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]
  count=0
  NoneFrames =[]
  for j in range(len(relative_keypoints)): # jth frame of window (train example)
        flag=1
        for k in range(len(relative_keypoints[j])):
            if relative_keypoints[j][k][0] is None or relative_keypoints[j][k][1] is None:
              count+=1
              NoneFrames.append(j)
              break
  if(count<=4):
        return ReplaceNonebyFrameAverage(relative_keypoints,NoneFrames)
  return None


In [19]:
numOfKeypoints=35

import numpy as np
import tensorflow as tf

# calculate the center of the object in each frame

def CentralisedKeypoints2(keypoints):
  frame_centers = []
  for frame_keypoints in keypoints:
    # assume the center is the mean of all keypoints
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))


  relative_keypoints = keypoints

  for j in range(len(relative_keypoints)):               #frame no j
    for k in range(len(relative_keypoints[j])):          # k keypoint of frame j
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]

    frames_keypoints=[]
    for j in range(len(relative_keypoints)):
        flag=1
        for k in range(len(relative_keypoints[j])):
          if relative_keypoints[j][k][0] is None or relative_keypoints[j][k][1] is None:
            flag=0
            break
        if(flag==1):
          frames_keypoints.append(relative_keypoints[j])

    if(len(frames_keypoints)>0):
      kk = [[0.0,0.0] for _ in range(numOfKeypoints)]
      for x in range (16-len(frames_keypoints)):
            frames_keypoints.append(kk)
      return frames_keypoints

  return None



In [20]:
numOfKeypoints = 35
def getKeyPointsforFrame(frame):
  img,results = mpPoseImg(frame)
  keypoints = [[None,None] for _ in range(numOfKeypoints)]
  if results.pose_landmarks == None:
    return keypoints
    
  landmarks = results.pose_landmarks
  for i in range(numOfKeypoints-2):
     mark = landmarks.landmark[i]
     keypoints[i][0]=mark.x
     keypoints[i][1]=mark.y
  #------------------------------------------------------------------------------------------------
  try:
    # MID HIP COORDINATES
      left_hip_index = 23
      right_hip_index = 24
      Mid_hip_x =  (keypoints[left_hip_index][0]+keypoints[right_hip_index][0])/2
      Mid_hip_y =(keypoints[left_hip_index][1]+keypoints[right_hip_index][1])/2
      # keypoints.append([Mid_hip_x,Mid_hip_y])
      keypoints[32][0]=Mid_hip_x
      keypoints[32][1]=Mid_hip_y
                          
  except:
      pass
      
  #------------------------------------------------------------------------------------------------
  try:
    # NECK COORDINATES
      left_shoulder_index = 11
      right_shoulder_index = 12
      Neck_x =  (keypoints[left_shoulder_index][0]+keypoints[right_shoulder_index][0])/2
      Neck_y =(keypoints[left_shoulder_index][1]+keypoints[right_shoulder_index][1])/2
      # keypoints.append([Neck_x,Neck_y])
      keypoints[33][0]=Neck_x
      keypoints[33][1]=Neck_y
                          
  except:
      pass

  #------------------------------------------------------------------------------------------------
     
  return keypoints

In [7]:
# imagePath='Images/test.jpg'
# img = cv2.imread(imagePath)
# getKeyPointsforFrame(img)

In [8]:
from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model

In [9]:
import pandas as pd

# read the data from the CSV file
data = pd.read_csv('PoseStream_train_data.csv')

# load the keypoints and labels into separate variables
# train_keypoints = list(data['keypoints'])
train_keypoints=[]
train_labels = list(data['label'])

numOfKeypoints = 35
import ast
for keypoints_str in data['keypoints']:
    keypoints_list = ast.literal_eval(keypoints_str)
    train_keypoints.append(keypoints_list)

In [10]:
# I=0
# for i in range(len(train_keypoints)):
#   F = 0
#   for j in range (16):
#     for k in range (34):
#       K=0
#       if(train_keypoints[i][j][k][0] is not None):
#         K+=1
#         if(K==34):
#            F+=1
#            if(F==16):
#              I+=1
#            print(I," ", F," ",K)          

In [21]:
new_train_keypoints = []
new_train_labels = []
for i in range(len(train_keypoints)):
  keypoints = CentralisedKeypointswithGlobalAverages(train_keypoints[i])
  if(keypoints is not None):
     new_train_keypoints.append(keypoints)
     new_train_labels.append(train_labels[i])


<ipython-input-18-a37f5a22170a>:6: RuntimeWarning: Mean of empty slice
  center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
<ipython-input-18-a37f5a22170a>:7: RuntimeWarning: Mean of empty slice
  center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])


[[-0.4656830046858106, -0.38754835554531647], [-0.5428800795759473, -0.36363014408520294], [-0.5499205206121717, -0.3865036053316934], [-0.5548696134771619, -0.4065239710467203], [-0.5533004019941602, -0.3642449659960611], [-0.567286512681416, -0.3884039206164224], [-0.5803257439817701, -0.4091951532023294], [-0.6243662450994764, -0.6422989053385599], [-0.6364987705435071, -0.6463098806994303], [-0.4458492611135755, -0.5656076354639872], [-0.4539896462644849, -0.5682848972933634], [-0.5373307679380689, -1.091018168415342], [-0.5343622182096753, -1.2176978273051127], [-0.555110535451344, 0.21749034694262914], [-0.5671646807874952, -0.08118679949215481], [-0.6247458670820508, 1.6833049373967306], [-0.6630792830671582, 1.0779189782483238], [-0.7900217984403882, 1.807440789256777], [-0.8206270251955303, 1.1526791172368185], [-0.8454910133566174, 1.7080933766705648], [-0.8184151862348827, 1.2006144838673727], [-0.7751399789537702, 1.66424486211368], [-0.749728432723454, 1.200578601871218], 

In [22]:
len(new_train_keypoints)

995

In [23]:
# train_videos = np.asarray(train_videos)
train_keypoints = np.asarray(train_keypoints)
train_labels = np.asarray(train_labels).astype('int32')

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(train_keypoints,train_labels,test_size=0.25,shuffle=True)

In [24]:
XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)

In [25]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 35, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300
23/23 [==============================] - 7s 306ms/step - loss: nan - accuracy: 0.0536 - val_loss: nan - val_accuracy: 0.0440
Epoch 2/300
23/23 [==============================] - 5s 203ms/step - loss: nan - accuracy: 0.0495 - val_loss: nan - val_accuracy: 0.0440
Epoch 3/300
23/23 [==============================] - 5s 202ms/step - loss: nan - accuracy: 0.0495 - val_loss: nan - val_accuracy: 0.0440
Epoch 4/300
23/23 [==============================] - 7s 310ms/step - loss: nan - accuracy: 0.0495 - val_loss: nan - val_accuracy: 0.0440
Epoch 5/300
 6/23 [======>.......................] - ETA: 3s - loss: nan - accuracy: 0.0417

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(XKey_test,yKey_test)

# [1.0725456476211548, 0.7861841917037964]  - None -> 0.0 , PoseStream1.h5
# [0.8455842733383179, 0.7861841917037964]  - None -> -1, PoseStream2.h5
# [1.0688899755477905, 0.7763158082962036]  - None -> Average, PoseStream3.h5 - all None frames not discarded

10/10 [==============================] - 1s 53ms/step - loss: 1.0689 - accuracy: 0.7763


[1.0688899755477905, 0.7763158082962036]

In [ ]:
# Save the model to a file
model.save('PoseStream3.h5')

In [ ]:


################### RUN THIS CELL


# Load the Drive helper and mount
from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model


import numpy as np
import pandas as pd
import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = './Videos'


labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107



train_videos=[]
train_labels=[]
import traceback
import random

num = 1

for i,folder in enumerate(labels):
    try:
      for video in os.listdir(data_path+'/'+folder):
        print(num)
        num+=1
        video = os.path.join(data_path+'/'+folder+'/'+video)
        # Open the video file
        cap = cv2.VideoCapture(video)

        # Get the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Select a random starting frame number
        start_frame = random.randint(0, total_frames - 16)

        # Set the number of frames to be selected
        num_frames = 16

        # Set the frame number to start with
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # Loop through the frames and save them
        frames = []
        for j in range(num_frames):
            ret, image = cap.read()
            if ret:
              frames.append(cv2.resize(image,(112,112)))

        # Release the video file
        cap.release()

        train_videos.append(frames)
        train_labels.append(i)

    except Exception:
      traceback.print_exc()
train_videos = np.asarray(train_videos)
train_labels = np.asarray(train_labels).astype('int64')

# import numpy as np
# arr_reshaped = train_videos.reshape(train_videos.shape[0], -1) #1151
# np.savetxt("dataset.txt", arr_reshaped)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_videos,train_labels,test_size=0.25,shuffle=True)

input_shape =(32,16,112,112,3)
output_shape = 20

tf.keras.regularizers.L1L2(
    l1=0.01, l2=0.1
)

RGBmodel = Sequential()

# model.add(RandomFlip("horizontal"))
# model.add(RandomRotation(0.1))
# RGBmodel.add(Rescaling(1.0 / 255,input_shape=input_shape[1:]))
RGBmodel.add(Conv3D(64,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same',input_shape=input_shape[1:]))
RGBmodel.add(MaxPooling3D(pool_size=(1, 2, 2),strides =(1,2,2)))    #maxPool-1

RGBmodel.add(Conv3D(128,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-2
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3a
RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5b

RGBmodel.add(ZeroPadding3D(padding=(0,1,1)))
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))
RGBmodel.add(Flatten())

RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l2'))
RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l1'))


RGBmodel.add(Dense(units = output_shape, activation = 'softmax'))


from keras.optimizers import SGD
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 400, validation_split = 0.2)


from keras.optimizers import SGD
opt = SGD(learning_rate=0.0001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2)

# Print the summary of the model
RGBmodel.summary()
RGBmodel.save('RGBmodel_whole.h5') 




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
# softmaxRGB = RGBmodel.predict(X_trainRGB)
# softmaxPose = Posemodel.predict(XKey_train)

In [ ]:
import numpy as np

# Combine the softmax outputs by taking the average of their values
combined_softmax = (softmaxRGB*0.7 + softmaxPose*0.3) 

# Generate some example ground truth labels
actualValues = Ytrain

# Convert the combined softmax probabilities to predicted labels
predicted_labels = np.argmax(combined_softmax)

# Compute the accuracy of the predictions
accuracy = np.mean(predicted_labels == actualValues)

# Print the results
print("Combined softmax probabilities:", combined_softmax)
print("Ground truth labels:", actualValues)
print("Predicted labels:", predicted_labels)
print("Accuracy:", accuracy)


In [ ]:
from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model

# This will prompt for authorization.
drive.mount('/MyDrive')

import numpy as np
import pandas as pd
import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = '../MyDrive/MyDrive/Dataset'


labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107

train_videos=[]
train_labels=[]
train_keypoints = []
import traceback
import random

num = 1

for i,folder in enumerate(labels):
  try:
    for video in os.listdir(data_path+'/'+folder):
      print(num)
      num+=1
      video = os.path.join(data_path+'/'+folder+'/'+video)
      # Open the video file
      cap = cv2.VideoCapture(video)

      # Get the total number of frames in the video
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

      # Select a random starting frame number
      start_frame = random.randint(0, total_frames - 16)

      # Set the number of frames to be selected
      num_frames = 16

      # Set the frame number to start with
      cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

      # Loop through the frames and save them
      frames = []
      keypoints = []
      for j in range(num_frames):
          ret, image = cap.read()
          if ret:
            frames.append(cv2.resize(image,(112,112)))
            kk = getKeyPointsforFrame(image)
            keypoints.append(kk)

      # Release the video file
      cap.release()


      train_videos.append(frames)

      train_keypoints.append(CentralisedKeypoints2(keypoints))
      train_labels.append(i)
      # if(num>2):
      #    break

  except Exception:
    traceback.print_exc()

In [ ]:
softmaxRGB = RGBmodel.predict(X_trainRGB)
softmaxPose = Posemodel.predict(XKey_train)

In [ ]:
import numpy as np

# Combine the softmax outputs by taking the average of their values
combined_softmax = (softmaxRGB*0.7 + softmaxPose*0.3) 

# Generate some example ground truth labels
actualValues = yKey_train

# Convert the combined softmax probabilities to predicted labels
predicted_labels = np.argmax(combined_softmax)

# Compute the accuracy of the predictions
accuracy = np.mean(predicted_labels == actualValues)

# Print the results
print("Combined softmax probabilities:", combined_softmax)
print("Ground truth labels:", actualValues)
print("Predicted labels:", predicted_labels)
print("Accuracy:", accuracy)



In [ ]:
labels=['Adhomukhasvanasana',
  'Ardhachakrasana',
  'Bhujangasana',
  'Dhanurasana',
  'Marjariasana',
  'Padahastasana',
  'Padmasana',
  'Pawanmuktasana',
  'Phalakasana',
  'Sarvangasana',
  'Sashankasana',
  'Setubandhasana',
  'Shavasana',
  'Tadasana',
  'Trikonasana',
  'Ustrasana',
  'Vakrasana',
  'Virbhadrasana1',
  'Virbhadrasana2',
  'Vrikshasana']
  

def predictYogaPose(video,keypoints):
  softmaxRGB = RGBmodel.predict(video)
  softmaxPose = Posemodel.predict(keypoints)
  combined_softmax = (softmaxRGB*0.7 + softmaxPose*0.3) 

  predicted_labels = np.argmax(combined_softmax)

  return labels[predicted_labels]




In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

data_path = './Videos'

labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107
labels

['Adhomukhasvanasana',
 'Ardhachakrasana',
 'Bhujangasana',
 'Dhanurasana',
 'Marjariasana',
 'Padahastasana',
 'Padmasana',
 'Pawanmuktasana',
 'Phalakasana',
 'Sarvangasana',
 'Sashankasana',
 'Setubandhasana',
 'Shavasana',
 'Tadasana',
 'Trikonasana',
 'Ustrasana',
 'Vakrasana',
 'Virbhadrasana1',
 'Virbhadrasana2',
 'Vrikshasana']

In [ ]:
RGBmodel = load_model('RGBStream.h5')
Posemodel = load_model('PoseStream.h5')

import cv2
cap = cv2.VideoCapture(0)

video = []

while True:
  ret, frame = cap.read()

  if not ret:
    break
  
  video.append(frame)

  if(len(video)==16):
    keypoints = []
    for image in video:
      keypoints.append(getKeyPointsforFrame(image))

    keypoints = CentralisedKeypoints2(keypoints)

    if(keypoints == None):
      print("video not visible")

    yogaPose = predictYogaPose(video,keypoints)



In [5]:
import numpy as np

# Create example window of frames
window = np.array([[[1, 2], [3, 4], [None,None], [5, 6]],
                   [[2, 3], [None,None], [4, 5], [6, 7]]] * 8)

# Mask out None values
# masked_window = np.ma.masked_invalid(window)

# Calculate average of valid frames
avg_frame = np.nanmean(window, axis=0)

print(avg_frame)


def windowMean(window):
  for i 


TypeError: ignored